# Lab 3 — ASR + LLM + TTS end-to-end
Este cuaderno reproduce el flujo sugerido por el profesor: grabar audio desde Colab, transcribirlo con Whisper, generar una respuesta breve con un LLM e inferir audio con Coqui TTS.


In [ ]:
!pip install openai-whisper # NO whisper
!pip install jiwer
!pip -q install transformers accelerate sentencepiece
!pip install coqui-tts  # usar coqui-tts (no el paquete "tts")


## Grabación de audio
Reutiliza el widget proporcionado en el notebook de referencia del profesor para asegurar compatibilidad con Colab.


In [ ]:
from IPython.display import Javascript, HTML, display, Audio
from google.colab import output
import base64, subprocess, uuid, os, pathlib

# @title Grabación de audio
# @markdown Lo necesitas para grabar audio desde Colab. Intenta mejorarlo.
def record(out_webm=None, out_wav=None, sr=16000, autoplay=False):
    if out_webm is None:
        out_webm = f"/content/rec_{uuid.uuid4().hex}.webm"
    if out_wav is None:
        out_wav = f"/content/rec_{uuid.uuid4().hex}.wav"

    js = Javascript(r"""
    async function recorderUIOnce(){
      const existing = document.getElementById('recorder-box');
      if (existing) { existing.remove(); }

      const box = document.createElement('div');
      box.id = 'recorder-box';
      box.style.cssText = 'padding:12px;margin:8px 0;border:1px solid #ddd;border-radius:10px;display:inline-flex;gap:8px;align-items:center;font-family:sans-serif';

      const dot = document.createElement('span');
      dot.style.cssText = 'width:10px;height:10px;border-radius:50%;background:#bbb';

      const startBtn = document.createElement('button');
      startBtn.textContent = 'Grabar';
      startBtn.style.cssText = 'padding:6px 10px';

      const stopBtn = document.createElement('button');
      stopBtn.textContent = 'Parar';
      stopBtn.style.cssText = 'padding:6px 10px';
      stopBtn.disabled = true;

      const msg = document.createElement('span');
      msg.textContent = 'Listo para grabar';
      msg.style.minWidth = '180px';

      box.append(dot, startBtn, stopBtn, msg);
      document.body.appendChild(box);

      let stream, rec, chunks = [];

      function setRec(on){
        dot.style.background = on ? '#e74c3c' : '#bbb';
        startBtn.disabled = on;
        stopBtn.disabled = !on;
        msg.textContent = on ? 'Grabando…' : 'Listo para grabar';
      }

      return await new Promise(async (resolve, reject) => {
        try {
          stream = await navigator.mediaDevices.getUserMedia({ audio:true });
          rec = new MediaRecorder(stream);
        } catch (err) {
          box.remove();
          reject('No se pudo acceder al micrófono: ' + err);
          return;
        }

        rec.ondataavailable = e => { if (e.data && e.data.size > 0) chunks.push(e.data); };

        rec.onstop = async () => {
          try {
            const blob = new Blob(chunks, {type:'audio/webm;codecs=opus'});
            const buf = await blob.arrayBuffer();
            const b64 = btoa(String.fromCharCode(...new Uint8Array(buf)));
            stream.getTracks().forEach(t => t.stop());
            box.remove();
            resolve(b64);
          } catch (err) {
            stream.getTracks().forEach(t => t.stop());
            box.remove();
            reject(err);
          }
        };

        startBtn.onclick = () => { chunks = []; rec.start(); setRec(true); };
        stopBtn.onclick = () => {
          if (rec && rec.state === 'recording') {
            rec.stop();
            setRec(false);
          }
        };
      });
    }
    """)
    display(js)

    b64 = output.eval_js("recorderUIOnce()")
    with open(out_webm, "wb") as f:
        f.write(base64.b64decode(b64))

    subprocess.run([
        "ffmpeg", "-y", "-i", out_webm, "-ac", "1", "-ar", str(sr), out_wav
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    try:
        os.remove(out_webm)
    except FileNotFoundError:
        pass

    display(Audio(filename=out_wav, autoplay=autoplay))
    return out_wav


In [ ]:
wav_path = record(out_wav="/content/input.wav", autoplay=True)


## Automatic Speech Recognition (ASR)


In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

import whisper, time
from jiwer import wer

MODEL = "turbo"   # opciones disponibles: tiny, base, small, medium, large, turbo
m = whisper.load_model(MODEL)
audio_in = "input.wav"

t0 = time.time()
out = m.transcribe(audio_in, language="en", task="transcribe", verbose=False)
t1 = time.time()

user_text = out["text"].strip()
print("ASR:", user_text)
print(f"Latency: {t1 - t0:.2f}s")


## LLM — Generación de respuesta corta


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "google/flan-t5-base"
tok = AutoTokenizer.from_pretrained(model_id)
mdl = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

pipe = pipeline("text2text-generation", model=mdl, tokenizer=tok)
prompt = f"Eres un asistente útil. Responde en 1-2 oraciones y sé directo. Pregunta: {user_text}"
resp = pipe(prompt, max_new_tokens=128)[0]["generated_text"].strip()
print("LLM:", resp)


## TTS — Síntesis con Coqui TTS


In [ ]:
from TTS.api import TTS

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

print(tts.speakers)

tts.tts_to_file(
    text=resp,
    # speaker_wav="ref_voice.wav",    # tu audio de referencia (5–10 s)
                                      # procura que sea audio de calidad
    speaker='Claribel Dervla',
    language="es",
    file_path="respuesta.wav"
)

display(Audio(filename="respuesta.wav", autoplay=True))
